# Figure 6

## Imports

In [ ]:
%%capture

# By these two lines, it will not be needed to restart kernel when applying change in a library
%load_ext autoreload
%autoreload 2

# import main
import dolfin  
import numpy   
import sympy    as sp
import matplotlib.pyplot as plt
from sympy.plotting import plot

from numpy import linspace
from sympy import lambdify
import sys
import os
import shutil

import dolfin_mech     as dmech
from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

from matplotlib import cm

import seeds
import geometry


## Defining Geometry and material parameters

In [ ]:
directory = "Hexagon"
parent_dir = "./Geometries"
path = os.path.join(parent_dir, directory) 
if os.path.exists(path):
    shutil.rmtree(path)
os.makedirs(path)

In [ ]:
%%capture
fname = "Geometries/Hexagon/hexagon_RVE1"

domain = 1
row = 1
DoI = 0.0
thickness = 0.092
shift_y = 0

seeds.semi_regular(DoI, row, domain)
geometry.voronoi(fname, thickness, row, domain, shift_y, seeds_remove=True)

mesh = dolfin.Mesh()
dolfin.XDMFFile(fname+"-mesh.xdmf").read(mesh)
Es = 1
nus = 0.499
mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":nus}}

coord = mesh.coordinates()
xmax = max(coord[:,0]); xmin = min(coord[:,0])
ymax = max(coord[:,1]); ymin = min(coord[:,1])
vol = (xmax - xmin)*(ymax - ymin)
dV = dolfin.Measure("dx",domain=mesh)
Phi_s0 = dolfin.assemble(1*dV)/vol

vertices = numpy.array([[xmin, ymin],
                     [xmax, ymin],
                     [xmax, ymax],
                     [xmin, ymax]])
bbox = [xmin, xmax, ymin, ymax]

## Kinematics

In [ ]:

alpha_x, alpha_y, alpha_xy, alpha_yx = sp.symbols('alpha_x alpha_y alpha_xy alpha_yx')
epsilon_x = sp.symbols('epsilon_x')
epsilon_y = sp.symbols('epsilon_y')
epsilon_xy = sp.symbols('epsilon_xy')
epsilon = sp.symbols('epsilon')
C_CM = sp.MatrixSymbol('C', 2, 2).as_explicit()

C00, C01, C10, C11 = sp.symbols('C00 C01 C10 C11')
C_CM = sp.Matrix([[C00, C01],
                  [C10, C11]])


p = sp.symbols('p')
I_C_CM   = sp.trace(C_CM) 
II_C_CM  = (sp.trace(C_CM)**2 - sp.trace(C_CM**2))/2 
III_C_CM = sp.det(C_CM) 
J_CM     = sp.sqrt(III_C_CM)
E_CM = (C_CM - sp.eye(2))/2 

C_CM_bar = J_CM**(-2/3)*C_CM
I_C_CM_bar   = sp.trace(C_CM_bar) 
II_C_CM_bar  = (sp.trace(C_CM_bar)**2 - sp.trace(C_CM_bar**2))/2 


F = sp.Matrix(
        [[alpha_x,    0  ],\
        [   0   , alpha_y]])

J = F.det()
C = F.T * F
E = (C - sp.eye(2))/2 

## Macroscopic model

In [ ]:
def macroscopic_model(mat_params):
    homo = dmech.HomogenizationProblem(
        dim=2,
        mesh=mesh,
        mat_params=mat_params["parameters"],
        vertices=vertices,
        vol=vol,
        bbox=bbox)

    lmbda, mu = homo.get_lambda_and_mu()
    beta = mu/2
    alpha = lmbda/4


    W_skel = beta * (I_C_CM - 2 - 2 * sp.ln(J_CM)) + alpha * (J_CM**2 - 1 - 2 * sp.ln(J_CM)) 
    Sigma_CM = 2*sp.diff(W_skel, C_CM) - p * J_CM * C_CM.inv()

    Sigma = Sigma_CM.subs(list(zip(C_CM, C))).doit().as_explicit() 
    sigma = F * Sigma * F.T / J 
    sigma = sigma.subs(alpha_x, epsilon + 1).subs(alpha_y, epsilon + 1)
    # sigma = sigma.subs(alpha_x, sp.sqrt(2*epsilon + 1)).subs(alpha_y, sp.sqrt(2*epsilon + 1))


    Ter_Sigma = 2*sp.diff(W_skel, C_CM)      
    Ter_Sigma = Ter_Sigma.subs(list(zip(C_CM, C))).doit().as_explicit() 
    Ter_Sigma = Ter_Sigma.subs(alpha_x, epsilon + 1).subs(alpha_y, epsilon + 1)
    # Ter_Sigma = Ter_Sigma.subs(alpha_x, sp.sqrt(2*epsilon + 1)).subs(alpha_y, sp.sqrt(2*epsilon + 1))

    return sigma, Ter_Sigma

sigma_macro, sigma_ter_macro = macroscopic_model(mat_params)

## Linear model

In [ ]:
homo = dmech.HomogenizationProblem(dim=2,
                mesh=mesh,
                mat_params=mat_params["parameters"],
                vertices=vertices,
                vol=vol,
                bbox=bbox)
lmbda_tilde, mu_tilde = homo.get_lambda_and_mu()
kappa_tilde = homo.get_kappa()
beta = mu_tilde/2
alpha = lmbda_tilde/4

sigma_lin =  2 * lmbda_tilde * epsilon + 2 * mu_tilde * epsilon

## Global response of the micromechanical model is a function of:
* material parameters
* $\epsilon_{xx}, \epsilon_{yy}$ $\longrightarrow$ macroscopic strain
* If no macroscopic strain is applied, `Macroscopic_strain` = <span style="color:lightblue">None</span>
* sigma_bar is the applied stress, otherwise:
\begin{equation}
    \sigma = \begin{bmatrix}
                0 & 0\\
                0 & 0
             \end{bmatrix}
\end{equation}

<!-- * $\gamma$ is the surface tension coefficient, otherwise it is equal to zero -->

In [ ]:
def global_response(mesh, mat_params, eps_xx, eps_yy, pf, Macroscopic_strain, sigma_bar, gamma):


    dim = 2
    bcs = "pbc"
    step_params = {"dt_ini":1e-1, "dt_min":1e-3}


    res_folder = sys.argv[0][:-3]
    res_basename  = sys.argv[0][:-3]

    res_basename = res_folder+"/"+res_basename
    verbose=1

    ################################################################### Mesh ###


    coord = mesh.coordinates()
    xmax = max(coord[:,0]); xmin = min(coord[:,0])
    ymax = max(coord[:,1]); ymin = min(coord[:,1])
    vol = (xmax - xmin)*(ymax - ymin)
    dV = dolfin.Measure("dx",domain=mesh)

    vertices = numpy.array([[xmin, ymin],
                         [xmax, ymin],
                         [xmax, ymax],
                         [xmin, ymax]])

    tol = 1E-8  
    vv = vertices
    a1 = vv[1,:]-vv[0,:] # first vector generating periodicity
    a2 = vv[3,:]-vv[0,:] # second vector generating periodicity
    # check if UC vertices form indeed a parallelogram
    assert numpy.linalg.norm(vv[2, :]-vv[3, :] - a1) <= tol
    assert numpy.linalg.norm(vv[2, :]-vv[1, :] - a2) <= tol
    bbox = [xmin, xmax, ymin, ymax]

    ################################################## Subdomains & Measures ###

    
    xmin_sd = dolfin.CompiledSubDomain("near(x[0], x0, tol) && on_boundary", x0=xmin, tol=tol)
    xmax_sd = dolfin.CompiledSubDomain("near(x[0], x0, tol) && on_boundary", x0=xmax, tol=tol)
    ymin_sd = dolfin.CompiledSubDomain("near(x[1], x0, tol) && on_boundary", x0=ymin, tol=tol)
    ymax_sd = dolfin.CompiledSubDomain("near(x[1], x0, tol) && on_boundary", x0=ymax, tol=tol)


    xmin_id = 1
    xmax_id = 2
    ymin_id = 3
    ymax_id = 4
    if (dim==3): zmin_id = 5
    if (dim==3): zmax_id = 6

    boundaries_mf = dolfin.MeshFunction("size_t", mesh, mesh.topology().dim()-1) # MG20180418: size_t looks like unisgned int, but more robust wrt architecture and os
    boundaries_mf.set_all(0)

    ################################################################ Problem ###

    problem = dmech.MicroPoroHyperelasticityProblem(
        mesh=mesh,
        mesh_bbox=bbox,
        vertices=vertices,
        boundaries_mf=boundaries_mf,
        displacement_perturbation_degree=2,
        quadrature_degree=3,
        solid_behavior=mat_params,
        bcs=bcs)

    ################################################################ Loading ###

    Deltat = step_params.get("Deltat", 1.)
    dt_ini = step_params.get("dt_ini", 1.)
    dt_min = step_params.get("dt_min", 1.)
    dt_max = step_params.get("dt_max", 1.)
    k_step = problem.add_step(
        Deltat=Deltat,
        dt_ini=dt_ini,
        dt_min=dt_min,
        dt_max=dt_max)


    problem.add_surface_pressure_loading_operator(
        measure=problem.dS(0),
        P_ini=0., P_fin=pf,
        k_step=k_step)

    for k in range(dim):
        for l in range (dim):
            if (sigma_bar[k][l] is not None):
                problem.add_macroscopic_stress_component_constraint_operator(
                    i=k, j=l,
                    sigma_bar_ij_ini=0.0, sigma_bar_ij_fin=sigma_bar[k][l],
                    pf_ini=0.0, pf_fin=pf,
                    k_step=k_step)

    if (Macroscopic_strain is not None):
        problem.add_macroscopic_stretch_component_penalty_operator(
            i=0, j=0,
            U_bar_ij_ini=0.0, U_bar_ij_fin=eps_xx,
            pen_val=1e6,
            k_step=k_step)
        problem.add_macroscopic_stretch_component_penalty_operator(
            i=1, j=1,
            U_bar_ij_ini=0.0, U_bar_ij_fin=eps_yy,
            pen_val=1e6,
            k_step=k_step)

    ################################################################# Solver ###

    solver = dmech.NonlinearSolver(
        problem=problem,
        parameters={
            "sol_tol":[1e-6]*len(problem.subsols),
            "n_iter_max":32},
        relax_type="constant",
        write_iter=0)

    integrator = dmech.TimeIntegrator(
        problem=problem,
        solver=solver,
        parameters={
            "n_iter_for_accel":4,
            "n_iter_for_decel":16,
            "accel_coeff":2,
            "decel_coeff":2},
        print_out=res_basename*verbose,
        print_sta=res_basename*verbose,
        write_qois=res_basename+"-qois",
        write_qois_limited_precision=1,
        write_sol=res_basename*verbose)

    success = integrator.integrate()
    assert (success),\
        "Integration failed. Aborting."

    integrator.close()
    

    for operator in problem.operators: 
        if hasattr(operator, "material"):
            material = operator.material
            break
    
    Vs0 = problem.mesh_V0
    U_bar = problem.get_macroscopic_stretch_subsol().func.vector().get_local().reshape((2,2))
    F_bar = U_bar + numpy.eye(2)
    J_bar = numpy.linalg.det(F_bar)
    C_bar = F_bar.T * F_bar
    E_bar = 0.5*(C_bar - numpy.eye(2))
    v = J_bar * vol
    vs = dolfin.assemble(problem.kinematics.J * problem.dV)
    vf = v - vs
    
    Phi_s = vs/vol


    
    sigma_tot_xx = float((dolfin.assemble(material.sigma[0,0] * problem.kinematics.J * dV) - vf * pf *dolfin.Identity(2)[0,0])/v)
    sigma_tot_yy = float((dolfin.assemble(material.sigma[1,1] * problem.kinematics.J * dV) - vf * pf *dolfin.Identity(2)[1,1])/v)

    U_bar_x = problem.get_macroscopic_stretch_subsol().func.vector().get_local()[0]
    U_bar_y = problem.get_macroscopic_stretch_subsol().func.vector().get_local()[3]
    U_bar_xy = problem.get_macroscopic_stretch_subsol().func.vector().get_local()[1]
    
    sigma = [[sigma_tot_xx, 0.],
             [0., sigma_tot_yy]]
    
    Sigma = J_bar * numpy.linalg.inv(F_bar)* sigma * numpy.linalg.inv(F_bar.T)
    Sigma_x = float(Sigma[0, 0])

    Ter_stress = Sigma_x + pf*J_bar*numpy.linalg.inv(C_bar)[0,0]

    normalized_volume = J_bar

    return normalized_volume, sigma_tot_xx, sigma_tot_yy, Ter_stress, Phi_s, U_bar_x, U_bar_y, U_bar_xy


## Macro-micro model comparision 

In [ ]:
sigma_bar = [[0., 0.],
             [0., 0.]]

## $\sigma - E$

In [ ]:
E_x_lst = []
sigma_Eps_small = []
eps_glob_small = []

for i in range(21):
    eps = i/20
    eps_glob_small.append(eps)
    glob_resp_ = global_response(mesh=mesh, mat_params=mat_params, eps_xx=eps, eps_yy=eps, pf=0., Macroscopic_strain=1, sigma_bar=sigma_bar, gamma=None)
    sigma_Eps_small.append(glob_resp_[1])
    E_x_lst.append(glob_resp_[5])

In [ ]:
lam_lin = lambdify(epsilon, sigma_lin, modules=['numpy'])
lam_small = lambdify(epsilon, sigma_macro.subs(p, 0)[0,0], modules=['numpy'])

epsilon_vals = linspace(0, 1, 100)
sigma_small = lam_small(epsilon_vals)


sigma_vals_small = lam_small(epsilon_vals)
sigma_vals = lam_lin(epsilon_vals)


plt.figure()
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel(r'$E_x, E_y~()$', fontsize=16)
plt.ylabel(r'$\sigma~(kPa)$', fontsize=16)
plt.plot(epsilon_vals, sigma_vals, '#005824')
plt.plot(eps_glob_small, sigma_Eps_small, '#084594')
plt.plot(epsilon_vals, sigma_vals_small, '#99000D')
plt.legend(['Linear model '+ r'$\tilde{\sigma}_{xx},~\tilde{\sigma}_{yy}$', 'Microscopic model '+ r'$\tilde{\sigma}_{xx},~\tilde{\sigma}_{yy}$', 'Macroscopic model '+ r'$\bar{\sigma}_{xx},~\bar{\sigma}_{yy}$'])
plt.xlim(0, 0.5)
plt.ylim(0, 0.15)
# plt.savefig('macro-micro_small.pdf',bbox_inches='tight')
plt.show()

## $\sigma - p_f$

In [ ]:
p_lst = []
sigma_xx_lst = []
sigma_bar = [[0., 0.],
             [0., 0.]]

for i in range(11):
    p_ = i/10
    p_lst.append(p_)
    sigma_xx_lst.append(global_response(mesh=mesh, mat_params=mat_params, eps_xx=0.0, eps_yy=0.0, pf=p_, Macroscopic_strain=1, sigma_bar=sigma_bar, gamma=None)[1])

In [ ]:
sigma_lin_p = -p

lam_in = lambdify(p, sigma_lin_p, modules=['numpy'])
lam = lambdify(p, sigma_macro[0,0].subs(epsilon, 0), modules=['numpy'])

p_vals = linspace(0, 1, 100)
sigma_xx_lam_lin = lam_in(p_vals)
sigma_xx_lam = lam(p_vals)

plt.figure()
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel(r'$p_f~(kPa)$', fontsize=16)
plt.ylabel(r'$\sigma~(kPa)$', fontsize=16)
plt.plot(p_vals, sigma_xx_lam_lin, '#005824')
plt.plot(p_lst, sigma_xx_lst, '#084594')
plt.plot(p_vals, sigma_xx_lam, '#99000D')
plt.legend(['Linear model '+ r'$\tilde{\sigma}_{xx},~\tilde{\sigma}_{yy}$', 'Microscopic model '+ r'$\tilde{\sigma}_{xx},~\tilde{\sigma}_{yy}$', 'Macroscopic model '+ r'$\bar{\sigma}_{xx},~\bar{\sigma}_{yy}$'])

plt.xlim(0, 0.5)
plt.ylim(-0.5, 0)
# plt.savefig('pf_sigma_incompressible.pdf', bbox_inches='tight')
plt.show()